In [2]:
import pandas as pd

from Pricing4API.plan import Plan
from Pricing4API.pricing import Pricing
from Pricing4API.optimal_subscription import *

s_second = 1
s_minute= 60
s_hour = 3600
s_day = 3600 * 24
s_month = 3600 * 24 * 30

ListSendGrid=[]

ListSendGrid.append(Plan('Basic', (0.0, s_month, 0.001), (10, s_second), [(1500, s_month)]))
ListSendGrid.append(Plan('Pro', (9.95, s_month, 0.001), (10, s_second), [(40000, s_month)], 10))
ListSendGrid.append(Plan('Ultra', (79.95, s_month, 0.00085), (10, s_second), [(100000, s_month)], 10))
ListSendGrid.append(Plan('Mega', (199.95, s_month, 0.00005), (50, s_second), [(300000, s_month)], 50))

PricingSendGrid = Pricing('SendGrid', ListSendGrid, 'mails')


In [ ]:
import pandas as pd

from Pricing4API.plan import Plan
from Pricing4API.pricing import Pricing


s_second = 1
s_minute= 60
s_hour = 3600
s_day = 3600 * 24
s_month = 3600 * 24 * 30

ListDBLP=[]
ListDBLP.append(Plan('Pro', (9.99, 1, None), (1, 2*s_second), [(20, s_minute), (1000, s_hour)]))

PricingDBLP = Pricing('DBLP', ListDBLP, 'queries')
PricingDBLP.link_plans()

In [ ]:
##Problema de la mochila. ¿A qué planes debo suscribirme para minimizar el precio para consumir n peticiones en un tiempo determinado?

num_correos = 15000
time = s_minute*20
resultado_combinacion, mejor_precio = get_optimal_subscription(PricingSendGrid.plans, num_correos, time)



In [ ]:
#1 ¿Cuantas requests se pueden hacer en un periodo de 1 semana para el Pricing de Sengrid?

for plan in PricingDBLP.plans:
    
    print("El numero de requests para el plan", plan.name, "de SendGrid es de:", plan.available_capacity('38s', len(plan.limits) - 1),"requests en")
    print("El numero de requests para el plan", plan.name, "de SendGrid es de:", plan.available_capacity('59s', len(plan.limits) - 1),"requests en")
    print("El numero de requests para el plan", plan.name, "de SendGrid es de:", plan.available_capacity('60s', len(plan.limits) - 1),"requests en")

In [ ]:
#2 ¿Cuánto tiempo necesita cada plan para hacer el numero máximo de request posibles en un instante t?
for plan in PricingDBLP.plans:
    print("El tiempo que tarda el plan", plan.name, "de", plan.name, "en consumir", plan.available_capacity('38s', len(plan.limits) - 1), "requests es de:", format_time(plan.min_time(plan.available_capacity('38s', len(plan.limits) - 1))))

In [ ]:
#3 ¿Cuánto tiempo estaría cada plan sin consumir desde que se ha alcanzado la capacidad máxima en un instante t?
tiempo=38
    
for plan in PricingDBLP.plans: 
    period_q = plan.limits[1][1]
    wastage_threshold = plan.compute_t_ast()
    print(f"El tiempo que estaría el plan {plan.name} de PricingDBLP sin consumir es de: {tiempo+(period_q-wastage_threshold[1])}")

In [ ]:
# 4: ¿Es posible enviar 40000 correos en 1 hora (asumiendo los limites de suscripción indicados en el pricing)?  
# Este codgio responde a la pregunta en conjunto para un pricing

def calcular_suscripciones(plans, num_correos, tiempo):
    suscripciones_utilizadas = []
    total_suscripciones = 0
    total_precio = 0
    
    for plan in plans:
        capacidad_en_tiempo = plan.max_capacity(tiempo)
        suscripciones_disponibles = plan.max_number_of_subscriptions
        
        suscripciones_necesarias = math.ceil(num_correos / capacidad_en_tiempo)
        suscripciones_a_utilizar = min(suscripciones_necesarias, suscripciones_disponibles)
        
        suscripciones_utilizadas.append((plan.name, suscripciones_a_utilizar))
        total_suscripciones += suscripciones_a_utilizar
        num_correos -= capacidad_en_tiempo * suscripciones_a_utilizar
        total_precio += suscripciones_a_utilizar * plan.price
        
        if num_correos <= 0:
            break
    
    return suscripciones_utilizadas, total_suscripciones, total_precio

# Llamada a la función con los planes definidos, el número de correos a enviar y el tiempo en segundos
num_correos = 1312
tiempo = 3600
resultado_suscripciones, total_suscripciones, total_precio = calcular_suscripciones(PricingSendGrid.plans, num_correos, tiempo)

print(f"Se necesitan las siguientes suscripciones para realizar {num_correos} requests en {format_time(tiempo)}:")
for plan, suscripciones in resultado_suscripciones:
    print(f"{suscripciones} suscripción del plan {plan}")

print(f"Total de suscripciones necesarias: {total_suscripciones}")
print(f"Precio total de suscripciones: {total_precio}")
